# Multi Layer Perceptron

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, recall_score

In [2]:
data=pd.read_csv("/content/data1.csv")
data.head()

,x_1,x_2,y
0,1.70600,2.8859,1
1,-0.70611,3.5617,-1
2,3.42530,1.9469,1
3,2.53570,1.5779,1
4,1.68960,3.4147,-1


In [3]:
data.describe()

,x_1,x_2,y
count,400.000000,400.000000,400.000000
mean,2.631745,2.657470,-0.020000
std,1.030100,1.002468,1.001052
min,-0.706110,-0.460070,-1.000000
25%,1.955625,1.963600,-1.000000
50%,2.657900,2.669150,-1.000000
75%,3.268950,3.339475,1.000000
max,6.314200,5.309100,1.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_1     400 non-null    float64
 1   x_2     400 non-null    float64
 2   y       400 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 9.5 KB


In [5]:
data_path="/content/data1.csv"
data_np=np.loadtxt(data_path, delimiter=",", skiprows=1,dtype=np.float32)

data_np

array([[ 1.706  ,  2.8859 ,  1.     ],
       [-0.70611,  3.5617 , -1.     ],
       [ 3.4253 ,  1.9469 ,  1.     ],
       ...,
       [ 3.0927 ,  4.2169 , -1.     ],
       [ 3.81   ,  3.2415 ,  1.     ],
       [ 4.7261 ,  2.8226 ,  1.     ]], dtype=float32)

In [6]:
data_t=torch.from_numpy(data_np)
data_t.shape

torch.Size([400, 3])

In [7]:
400*(80/100)

320.0

In [8]:
x, y = data_t[:,:-1],data_t[:,-1].long()
x.shape,y.shape

(torch.Size([400, 2]), torch.Size([400]))

In [9]:
y[y==-1]=0

In [10]:
x_train, y_train, x_test, y_test = x[:320], y[:320], x[320:], y[320:] #80% training data
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([320, 2]),
 torch.Size([320]),
 torch.Size([80, 2]),
 torch.Size([80]))

In [11]:
model=nn.Sequential(
    nn.Linear(2,32),
    nn.ReLU(),
    nn.Linear(32,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,2),
)

loss_fn=nn.CrossEntropyLoss()

def training_loop(n_epochs, loss_fn, x_train, y_train, model, optimizer):
  for i in range(n_epochs):
    loss=0
    for j in range(len(x_train)):
      y_pred=model(x_train[j].unsqueeze(0))
      train_loss=loss_fn(y_pred,y_train[j].unsqueeze(0))

      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()

      loss+=train_loss.item()

    print(f"Epoch: {i}, Train_loss: {loss/len(x_train)}")
  return [i for i in model.parameters()]


In [12]:
[i for i in model.parameters()]

[Parameter containing:
 tensor([[ 3.2282e-01, -1.3123e-01],
         [ 2.2465e-01, -3.8231e-01],
         [ 6.2280e-01, -3.6282e-01],
         [-2.5608e-04,  3.3637e-01],
         [ 4.8316e-01,  1.3894e-01],
         [-5.4403e-01,  2.3212e-01],
         [ 4.0116e-01, -3.3637e-01],
         [ 2.5753e-01, -1.8688e-01],
         [ 3.2374e-02,  2.7381e-01],
         [ 2.3240e-01,  5.1585e-01],
         [-7.3802e-02, -6.7661e-01],
         [-5.9686e-01,  2.3747e-01],
         [-4.2456e-01, -4.1155e-01],
         [ 9.0875e-02,  4.4336e-01],
         [-2.1319e-01, -4.7482e-01],
         [-3.4543e-01,  5.7770e-01],
         [-6.2345e-01, -2.4158e-02],
         [ 3.5032e-01, -6.6019e-02],
         [-1.7431e-01, -4.3133e-01],
         [ 1.5374e-01, -4.6471e-01],
         [-4.8850e-01,  1.7715e-01],
         [ 1.0945e-01, -6.3550e-01],
         [-1.1268e-01,  1.0047e-01],
         [-8.2393e-02,  3.7199e-01],
         [-4.6221e-01, -1.0258e-01],
         [ 2.8297e-01, -4.4061e-01],
         [ 2.68

In [13]:
x_train.shape

torch.Size([320, 2])

In [14]:
model_params = training_loop(
                    n_epochs=110,
                    loss_fn=loss_fn,
                    x_train=x_train,
                    y_train=y_train,
                    model=model,
                    optimizer=optim.SGD(model.parameters(),lr=1e-2)
                )

Epoch: 0, Train_loss: 0.5488297871721443
Epoch: 1, Train_loss: 0.37485038585873554
Epoch: 2, Train_loss: 0.36471895257709547
Epoch: 3, Train_loss: 0.35358691323817765
Epoch: 4, Train_loss: 0.3457909499313246
Epoch: 5, Train_loss: 0.3421940061278292
Epoch: 6, Train_loss: 0.337738693018764
Epoch: 7, Train_loss: 0.3338821177072532
Epoch: 8, Train_loss: 0.32979058276469003
Epoch: 9, Train_loss: 0.326491478789103
Epoch: 10, Train_loss: 0.3230674691381864
Epoch: 11, Train_loss: 0.31986301808719875
Epoch: 12, Train_loss: 0.3158145100722322
Epoch: 13, Train_loss: 0.312838293081586
Epoch: 14, Train_loss: 0.3098300737226964
Epoch: 15, Train_loss: 0.3066447402336053
Epoch: 16, Train_loss: 0.3041069613493164
Epoch: 17, Train_loss: 0.30170950179162903
Epoch: 18, Train_loss: 0.3000547662639292
Epoch: 19, Train_loss: 0.2986771714669885
Epoch: 20, Train_loss: 0.29744173447834327
Epoch: 21, Train_loss: 0.2961086954717757
Epoch: 22, Train_loss: 0.2951375270902645
Epoch: 23, Train_loss: 0.294013007145258

In [15]:
print(model)

Sequential(
  (0): Linear(in_features=2, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=2, bias=True)
)


In [16]:
model_params

[Parameter containing:
 tensor([[ 0.3065, -0.4044],
         [ 0.3421, -0.3649],
         [ 1.6660, -1.5260],
         [-0.3256,  0.6332],
         [ 0.6604, -0.2417],
         [-1.3981,  1.1883],
         [ 1.0115, -0.9550],
         [ 0.5937, -0.5337],
         [-0.2429,  0.5823],
         [-0.2187,  0.5995],
         [-0.1594, -0.8508],
         [-0.6292,  0.2446],
         [-0.4246, -0.4116],
         [-0.1900,  0.3037],
         [-0.2132, -0.4748],
         [-1.7731,  1.5489],
         [-0.6234, -0.0242],
         [ 0.6973, -0.5290],
         [-0.1692, -0.6419],
         [ 0.1653, -0.5634],
         [-0.4778,  0.2647],
         [ 0.1397, -0.7451],
         [-0.1972,  0.0744],
         [-1.0379,  1.0425],
         [-0.4622, -0.1026],
         [ 1.0558, -0.9954],
         [ 0.3029, -0.3124],
         [-0.5266, -0.5464],
         [-0.7067,  0.3430],
         [-0.4465,  0.4768],
         [ 1.5367, -1.4694],
         [ 0.5856, -0.4378]], requires_grad=True),
 Parameter containing:
 ten

In [17]:
#w_1= 0.0467  w_2=0.0506

In [18]:
print(model[6].weight, model[6].bias)

Parameter containing:
tensor([[ 0.0425, -0.2884, -0.2890, -0.0929, -0.2422,  0.0688,  0.3548,  0.0018,
         -0.4604, -0.0749, -0.1123,  0.0403,  0.1914,  0.0655,  0.1220, -0.0103,
         -0.0353,  0.3231,  0.0404, -0.2319, -0.3313,  0.2910,  0.0662, -0.1590,
          0.2052, -0.0565, -0.0916,  0.3846,  0.4673,  0.2351, -0.0368,  0.4744],
        [ 0.0739,  0.3221,  0.1009,  0.0719,  0.0296, -0.1580, -0.4834, -0.0299,
          0.4853,  0.1138, -0.1122,  0.0021, -0.4520,  0.0454,  0.0937,  0.0889,
         -0.1241, -0.1772, -0.1644,  0.1290,  0.3531, -0.1581,  0.0639,  0.0961,
         -0.0704,  0.0193, -0.0013, -0.4204, -0.2731, -0.2146, -0.1187, -0.5129]],
       requires_grad=True) Parameter containing:
tensor([-0.2944,  0.3650], requires_grad=True)


In [19]:
len(y_test)

80

In [20]:
y_pred=[]
for i in range(len(y_test)):
  _,pred=torch.max(model(x_test[i].unsqueeze(0)),dim=1)
  y_pred.append(pred.item())
len(y_pred)

80

In [21]:
y_pred=np.array(y_pred)
y_target=y_test.numpy()

y_pred.shape,y_target.shape

((80,), (80,))

In [22]:
acc= accuracy_score(y_target, y_pred)
prec = precision_score(y_target, y_pred)
f1 =f1_score(y_target, y_pred)
r=recall_score(y_target,y_pred)
cm = confusion_matrix(y_target, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall Score: {r:.4f}")
print("Confusion Matrix:\n", cm)

Accuracy: 0.9625
Precision: 0.9348
F1 Score: 0.9663
Recall Score: 1.0000
Confusion Matrix:
 [[34  3]
 [ 0 43]]


# Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score

In [24]:
data_path="/content/data1.csv"

data_np=np.loadtxt(
    data_path, delimiter=",", dtype=np.float32,skiprows=1
)

data_np.shape

(400, 3)

In [25]:
x=data_np[:,:-1]
y=data_np[:,-1:]
y[y==-1],y[y==-1]=0,0

x.shape, y.shape

((400, 2), (400, 1))

In [26]:
x_train, y_train, x_test, y_test=x[:320], y[:320], x[320:], y[:320]
y_train,y_test=y_train.squeeze().astype(int), y_test.squeeze().astype(int)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((320, 2), (320,), (80, 2), (320,))

In [27]:
lg=LogisticRegression()
lg.fit(x_train, y_train)
y_pred=lg.predict(x_test)

In [28]:
y_pred

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [29]:
acc= accuracy_score(y_target, y_pred)
prec=precision_score(y_target, y_pred)
f1=f1_score(y_target, y_pred)
r=recall_score(y_target,y_pred)
cm=confusion_matrix(y_target, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall Score: {r:.4f}")
print("Confusion Matrix:\n", cm)

Accuracy: 0.9875
Precision: 0.9773
F1 Score: 0.9885
Recall Score: 1.0000
Confusion Matrix:
 [[36  1]
 [ 0 43]]
